In [39]:
parent_folder = "/home/cpu11453/workplace/experiment/lightgbm_experiment/data"
train_path = parent_folder+"/"+"binary.train.txt"
test_path = parent_folder + "/" + "binary.test.txt"
mmlspark_pred_path = parent_folder + "/" + "pred_100" + "/" +"part-00000-827eb676-c758-46e5-82a2-6258d255d20e-c000.csv"

In [36]:
import numpy as np
import lightgbm as lgb
import pandas as pd
import os
from sklearn.metrics import mean_squared_error, accuracy_score

In [37]:
train_df = pd.read_csv(train_path, sep="\t", header=None)
test_df = pd.read_csv(test_path, sep="\t", header=None)

In [38]:
train_df.head()

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,1,0.869,-0.635,0.226,0.327,-0.690,0.754,-0.249,-1.092,0.000,...,-0.010,-0.046,3.102,1.354,0.980,0.978,0.920,0.722,0.989,0.877
1,1,0.908,0.329,0.359,1.498,-0.313,1.096,-0.558,-1.588,2.173,...,-1.139,-0.001,0.000,0.302,0.833,0.986,0.978,0.780,0.992,0.798
2,1,0.799,1.471,-1.636,0.454,0.426,1.105,1.282,1.382,0.000,...,1.129,0.900,0.000,0.910,1.108,0.986,0.951,0.803,0.866,0.780
3,0,1.344,-0.877,0.936,1.992,0.882,1.786,-1.647,-0.942,0.000,...,-0.678,-1.360,0.000,0.947,1.029,0.999,0.728,0.869,1.027,0.958
4,1,1.105,0.321,1.522,0.883,-1.205,0.681,-1.070,-0.922,0.000,...,-0.374,0.113,0.000,0.756,1.361,0.987,0.838,1.133,0.872,0.808


In [14]:
y_train = train_df[0]
y_test = test_df[0]
X_train = train_df.drop(0, axis=1)
X_test = test_df.drop(0, axis=1)

In [44]:
y_test.value_counts()

1    272
0    228
Name: 0, dtype: int64

In [17]:
# create dataset for lightgbm
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

In [18]:
params = {
    'objective': 'binary',
    'metric': {'accuracy'},
}

In [19]:
gbm = lgb.train(params,
                lgb_train,
                valid_sets=lgb_eval)

In [30]:
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)

In [31]:
y_pred_binary = np.where(y_pred > 0.5, 1, 0)

In [34]:
accuracy_score(y_test,y_pred_binary)

0.752

In [40]:
y_pred_mmlspark = pd.read_csv(mmlspark_pred_path, sep="\t", header=None)

In [41]:
y_pred_mmlspark

,0,1
0,1,1.0
1,0,0.0
2,0,0.0
3,1,1.0
4,0,0.0
...,...,...
495,1,1.0
496,0,0.0
497,1,1.0
498,0,0.0


In [42]:
accuracy_score(y_pred_mmlspark[0],y_pred_mmlspark[1])

0.752